# Counting Stars

## Setup

In [20]:
#!/usr/bin/env python3

import os
import sys
import time
import pyopencl as cl
import numpy
from PIL import Image, ImageOps
from libs.util import *

In [21]:
# Suppress kernel caching.
os.environ["PYOPENCL_NO_CACHE"] = "1"
os.environ["PYOPENCL_CTX"] = "0"

In [22]:
IMG_PATH = "../images/behemoth-black-hole.jpg"
KERNEL_NAME_GREY = "kernel_grey"
KERNEL_NAME_THRESHOLD = "kernel_threshold"
KERNEL_NAME_STARS = "kernel_stars"

In [23]:
img = image_to_array(IMG_PATH)
(img_h, img_w, depth) = img.shape
print(f"Image dimensions: {img_h}x{img_w} with depth {depth}")
flat_img = img.reshape(img_h * img_w * depth).astype(numpy.uint32)

Image dimensions: 2219x2243 with depth 3


In [24]:
# Create the context, queue and program.
context = cl.create_some_context()
queue = cl.CommandQueue(context)

kernel_grey_code = open("../kernels/" + KERNEL_NAME_GREY + ".cl").read()
kernel_threshold_code = open("../kernels/" + KERNEL_NAME_THRESHOLD + ".cl").read()
kernel_stars_code = open("../kernels/" + KERNEL_NAME_STARS + ".cl").read()
program_grey = cl.Program(context, kernel_grey_code).build()
program_threshold = cl.Program(context, kernel_threshold_code).build()
program_stars = cl.Program(context, kernel_stars_code).build()


# Initialize the kernel.
kernel_grey = program_grey.kernel_grey
kernel_threshold = program_threshold.kernel_threshold
kernel_stars = program_stars.kernel_stars

## Grayscale

In [25]:
#set argument types
kernel_grey.set_scalar_arg_dtypes(
    [None, None,
        numpy.int32, numpy.int32, numpy.int32]
)

In [26]:
# Create the result image.
h_output_img = numpy.zeros(img_h * img_w * 1).astype(numpy.uint32)

# Create the buffers on the device.
d_input_img = cl.Buffer(
    context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=flat_img
)

d_output_img = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_output_img.nbytes)

In [27]:
print("Executing kernel grey...")
kernel_grey(
    queue,
    (img_h, img_w),
    None,
    d_input_img,
    d_output_img,
    img_w,
    img_h,
    depth
)
print("Done executing kernel grey.")

queue.finish()
cl.enqueue_copy(queue, h_output_img, d_output_img)
queue.flush()

Executing kernel grey...
Done executing kernel grey.


In [28]:
#save gray image
result_img = h_output_img.reshape(img_h, img_w)
save_image_grey(result_img, "../output/" + "output_" + KERNEL_NAME_GREY + ".png")

## Threshold

In [29]:
LOCAL_SIZE = 1
global_size = int((img_w * img_h)/LOCAL_SIZE)

In [30]:
kernel_threshold.set_scalar_arg_dtypes(
    [None, None,
        numpy.int32, numpy.int32]
)

In [31]:
d_input_img = cl.Buffer(
    context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_output_img
)
h_input_brightness = numpy.zeros(
    global_size).astype(numpy.uint32)

d_output_brightness = cl.Buffer(
    context, cl.mem_flags.READ_WRITE, h_input_brightness.nbytes)

In [32]:
print("Executing kernel threshold...")
kernel_threshold(
    queue,
    (global_size, 1),
    (LOCAL_SIZE, 1),
    d_output_img,
    d_output_brightness,
    img_w,
    img_h
)
print("Done executing kernel threshold.")
queue.finish()
cl.enqueue_copy(queue, h_input_brightness, d_output_brightness)
queue.flush()

Executing kernel threshold...
Done executing kernel threshold.


In [33]:
print("Brightness: ", h_input_brightness.sum())
threshold = 2 * h_input_brightness.sum()/(img_w * img_h)
threshold = math.floor(threshold)
print("the threshold is: ", threshold)

Brightness:  271672305
the threshold is:  109


## Stars

In [34]:
LOCAL_SIZE = 4
global_size = int((img_w * img_h)/LOCAL_SIZE)

In [35]:
kernel_stars.set_scalar_arg_dtypes(
    [None, None,
        numpy.int32, numpy.int32, numpy.int32]
)

In [36]:
d_input_img = cl.Buffer(
    context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_output_img
)
h_input_count = numpy.zeros(
    global_size).astype(numpy.uint32)
d_output_count = cl.Buffer(
    context, cl.mem_flags.READ_WRITE, h_input_brightness.nbytes)

In [37]:
print("Executing stars kernel...")
kernel_stars(
    queue,
    (global_size, 1),
    (LOCAL_SIZE, 1),
    d_input_img,
    d_output_count,
    threshold,
    img_h,
    img_w
)
print("Done executing stars kernel.")
queue.finish()
cl.enqueue_copy(queue, h_input_count, d_output_count)
queue.flush()

Executing stars kernel...
Done executing stars kernel.


In [38]:
print("Count: ", sum(h_input_count))

Count:  67419
